# Introduction

In this assignment, you will embark on the exciting task of constructing a spell checker utilizing the noisy channel model. Your mission involves the implementation of a segment dedicated to the noisy-channel model for spelling correction, complemented by the integration of diverse language models. During the testing phase, you will encounter sentences intentionally infused with a single typing error. The objective is to meticulously identify and rectify the error by selecting the correction that attains the highest likelihood under the noisy-channel model. Additionally, your language model will serve as the crucial prior in this correction process. The effectiveness of your spell checker will be assessed based on accuracy, calculated as the ratio of valid corrections to the total number of test sentences. Prepare to delve into the intricacies of language modeling for spell-checking.





# Preliminaries

In this assignment, you will be utilizing three essential class structures provided to you: `LexicalEntry`, `Sentence`, and `Corpus`. 
Before delving into the core tasks of the assignment, it is imperative to familiarize yourself with these foundational structures. Each class serves a distinct purpose in facilitating your understanding of lexical entries, sentence structures, and corpus organization.


In [2]:
from utils import LexicalEntry, Sentence, Corpus, SpellingResult, group_n_words
import collections
import math


## LexicalEntry

The `LexicalEntry` class represents a unit of information about a word and its potential error within the context of a spelling correction system. Here's a summary of its key functionalities:

- **Initialization:** The class is initialized with a correct `word` and an optional `error` word, which defaults to an empty string if not provided.
- **Fixing Error:** The `fixError` method creates a new `LexicalEntry` object with the same correct word but an empty error attribute, essentially fixing the error.
- **Error Checking:** The `hasError` method checks if the `LexicalEntry` object has an error, returning **True** if an error is present and **False** otherwise.
- **Validity Testing:** The `isValidTest` method determines if the error in the `LexicalEntry` is within an edit distance of **one** and contains no numerics or punctuation. It returns **True** if the conditions are met, and **False** otherwise.
- **String Representation:** The `__str__` method creates a string representation of the `LexicalEntry` object in the format **word (error)** if an error is present, and simply **word** if there is no error.

This class provides a comprehensive set of methods to manage and analyze lexical entries, making it a crucial component in the broader spell-checking system. It encompasses functionality for error correction, error checking, validity testing, and string representation of lexical entries.


### Usage examples

Let's instantiate a `LexicalEntry` object with the correct word **love** and an associated error word **lov**. 

In [3]:
lexical_entry = LexicalEntry("love", "lov")

Let's check if there is an error in that `LexicalEntry` object:

In [4]:
lexical_entry.hasError()

True

Let's check if the error is withing Edit distance of 1: 

In [5]:
lexical_entry.isValidTest()

True

Let's now print it using the `__str__` method to show the error:


In [ ]:
print(lexical_entry)

Finally, let's fix the error and print it again:

In [ ]:
lexical_entry_fixed = lexical_entry.fixError()
print(lexical_entry_fixed)

## Sentence

The `Sentence` class is designed to manage and manipulate a sequence of `LexicalEntry` instances, each representing a word with potential errors in a sentence. Here is a summary of its key methods:

- **Initialization:** The class is initialized with a list of `LexicalEntries` representing the words in a sentence. The default is an empty list if not provided.
- **Error and Correction Retrieval:** Methods such as `getErrorSentence` and `getCorrectSentence` return lists of strings containing the sentence with errors or corrections, respectively.
- **Correction Verification:** The `isCorrection` method checks if a given list of strings is a correction of the sentence.
- **Error Index Detection:** The `getErrorIndex` method returns the index of the first error in the sentence or -1 if there is no error.
- **Index-based Access and Modification:** Methods like `get` retrieve the `LexicalEntry` at a specified index, while `update` modifies the entry at a given index.
- **Sentence Cleaning:** The `cleanSentence` method creates a new sentence with all `LexicalEntry` instances having errors removed.
- **Empty Check:** The `isEmpty` method checks if the sentence is empty.
- **Appending and Length Calculation:** The `append` method adds a `LexicalEntry` to the sentence, and the `__len__` method returns the length of the sentence.
- **String Representation:** The `__str__` method generates a string representation of the `Sentence` object, combining the string representations of individual `LexicalEntry` instances in the sentence.


Overall, the `Sentence` class provides a robust set of methods for managing and analyzing sentences with potential spelling errors, making it a valuable component in the context of spelling correction systems.



### Usage examples

Let's instantiate a `Sentence` object with the sentence **"i study at Deakin University"** with a correct word **study** and an associated error word **stud**. . 

In [ ]:
lst = [LexicalEntry("i"), LexicalEntry("study", "stud"), LexicalEntry("at"), LexicalEntry("Deakin"), LexicalEntry("University")]
my_sentence = Sentence(lst)
# Print the sentence
print(my_sentence)


Let's first print the error sentence and the correct sentence:

In [ ]:
print(f'The incorrect sentence is: {my_sentence.get_error_sentence()}')
print(f'The correct sentence is: {my_sentence.get_correct_sentence()}')


The `getErrorIndex` method can be used to return the index of the first error in the sentence:

In [ ]:
print(f'The correct sentence is: {my_sentence.get_error_index()}')


We can check if another sentence is a correction to our sentence:

In [ ]:
# Example usage
print(my_sentence.is_correction(["i", "study", "at", "Deakin", "University"]))


You can use the `get` method to retrieve the `LexicalEntry` at a specified index and update it if needed using `update`:

In [ ]:
# Print LexicalEntry at index 0
print(my_sentence.get(0))
# Update it with I and print sentence
my_sentence.update(0, LexicalEntry('I'))
print(my_sentence)


Finally, let's clean out sentence from errors and return a clean sentence:

In [ ]:
my_fixed_sentence = my_sentence.clean_sentence()
print(my_fixed_sentence)


## Corpus

The `Corpus` class represents a collection of sentences from a dataset and provides methods for processing and generating test cases with eligible spelling errors. Here's a breakdown of its key methods:

- **Initialization:** The class is initialized with an optional filename pointing to the Holbrook dataset. If a filename is provided, the dataset is read and processed; otherwise, an empty corpus is created.
- **Reading and Processing Data:** The `read_corpus` method reads data from a file, processes it into a list of `Sentence` objects, and populates the corpus.
- **Processing Line:** The `processLine` method takes a line from the dataset, removes punctuation, converts to lowercase, and creates a `Sentence` object with `LexicalEntry` instances.
- **Generating Test Cases:** The `generateTestCases` method creates a list of sentences with exactly one eligible spelling error by iterating through the corpus and selecting appropriate words for testing.
- **Vocabulary Retrieval:** The `get_vocabulary` method retrieves the vocabulary from the corpus, consisting of unique words.

In summary, the Corpus class serves as a container for sentences, offering methods for data initialization, processing, test case generation, and vocabulary retrieval. It plays a crucial role in facilitating the handling and manipulation of linguistic data within the context of spelling correction and language modeling.

Let's first create our `Corpus` object: 

In [ ]:
my_corpus = Corpus('data/trainset.txt')


We now print a few sentences from it:

In [ ]:
for i in range(10):
    print(my_corpus.corpus[i])

# The Noisy Channel Spell Correction Model

The foundation of the noisy channel model lies in treating a misspelled word as if it underwent distortion while being transmitted through a noisy communication channel. The objective is to construct a model of this noisy channel and determine the true word by evaluating the likelihood of each candidate word given the observed misspelling. The Bayesian inference approach is employed, seeking the word $w$ that maximizes the conditional probability $P(w|x)$. Mathematically expressed as:

$$\hat{w} = \underset{w\in V}{\operatorname{argmax}} P(w|x) $$


this signifies choosing the word with the highest likelihood from the vocabulary $V$. Bayesian classification leverages Bayes' rule to transform this into $P(x|w)P(w) / P(x)$, and by dropping the denominator $P(x)$, we get  the following simplified formula:

$$\hat{w} = \underset{w\in V}{\operatorname{argmax}}  \overbrace{P(x|w)}^\text{Channel Model} \times  \underbrace{P(w)}_\text{Language Model} $$

In essence, the model computes the most probable word given an observed misspelling by multiplying the prior $P(w)$ (**Language Model**) and the likelihood $P(x|w)$ (**the Channel Model**). The noisy channel algorithm, is then applied to correct non-word spelling errors by ranking candidate corrections according to the previous Equation and selecting the highest-ranked one. This process involves evaluating the likelihood (**Channel Model**) $P(x|w)$ and the prior (**Language Model**) $P(w)$.



## Channel Model

The estimation of the likelihood  $P(x|w)$, referred to as the channel model or error model, is a crucial aspect of the noisy channel model. This model aims to capture the probability that a word  $w$ w will be mistyped, taking into account various factors. While a perfect model might consider factors like the typist's identity or handedness, a practical estimate can be obtained by examining the **Minimal Damerau-Levenshtein Edit Distance** between two strings, where edits are:

- Insertion
- Deletion
- Substitution
- Transposition of two adjacent letters

For instance, letters like **'m'** and **'n'** are often substituted due to both their phonetic similarity and their adjacency on the keyboard. A simple model might estimate  $P(acress∣across)$ by analyzing the frequency of the letter **'e'** being substituted for **'o'** in a large corpus of errors. To compute these probabilities systematically, a confusion matrix is employed, which lists the number of times one element was confused with another. Specifically, the file `data/count_1edit.txt` which provides counts for all single-edit spelling correction edits. For example, the line `da|d 13` suggests that the correction of **'d'** to **'da'** has been observed 13 times in the data, whereas `e|i 917` indicates that the substitution of **'e'** to **'i'** has been observed 917 times in the data.



### <span style="color:red"><b>Task 1</b></span>

Let's build our Channel Model and by building the model to calculate our likelihood $P(x|w)$:

In [ ]:
class ChannelModel:
    alphabet = "abcdefghijklmnopqrstuvwxyz"

    def __init__(self, edit_file, vocabulary):
        """
        Initializes the ChannelModel with an optional edit file and a corpus.

        Parameters:
        - edit_file (str): Path to the edit file.
        - vocabulary (set): The vocabulary used.
        """
        self.edit_file = edit_file
        self.edit_table = self.read_edit_table(self.edit_file)
        self.vocabulary = vocabulary

    def compute_edit_probabilities(self, word):
        """
        Computes p(x|word) edit model for all valid word x ** that are in the vocabulary**. 
        Returns a dictionary mapping x -> p(x|word).

        Parameters:
        - word (str): The input word.

        Returns:
        - dict: A dictionary mapping possible corrections to their conditional probabilities p(x|word).
        """        
        counts = {}

        # Considering  deletions
        counts.update(self.process_deletions(word))
        # Considering insertions
        counts.update(self.process_insertions(word))
        # Considering transpositions
        counts.update(self.process_transpositions(word))
        # Considering replacements
        counts.update(self.process_replacements(word))

        # normalize counts. sum over them all, divide each entry by sum.
        total = 0.0
        for a, b in counts.items():
            total += b
        # We set the probability of no correction to .9. Hence, self count is set to 9*total
        selfCount = max(9 * total, 1)
        counts[word] = selfCount
        total += selfCount
        probs = {}

        if total != 0.0:
            for a, b in counts.items():
                probs[a] = float(b) / total

        return probs

    def process_deletions(self, word):
        """
        This function processes deletions on a given word by checking if each resulting word, obtained by 
        deleting a single letter at a time, is present in the vocabulary. If a resulting word is found
        in the vocabulary, it calculates the edit count and returns a dictionary containing these 
        resulting words along with their corresponding edit counts.

        Parameters:
        - word: The input word for which deletions need to be processed.

        Returns:
        - dict: A dictionary containing the resulting words after deletion and their corresponding edit counts.
        """        
        counts = {}
        ## START YOU CODE HERE
        pass
        ## END
        return counts

    def process_insertions(self, word):
        """
        This function processes insertions on a given word by checking if each resulting word, obtained by 
        inserting a single letter at a time, is present in the vocabulary. If a resulting word is found
        in the vocabulary, it calculates the edit count and returns a dictionary containing these 
        resulting words along with their corresponding edit counts.

        Parameters:
        - word: The input word for which insertions need to be processed.

        Returns:
        - dict: A dictionary containing the resulting words after insertion and their corresponding edit counts.
        """   
        counts = {}
        ## START YOU CODE HERE
        pass
        ## END
        return counts

    def process_transpositions(self, word):
        """
        This function processes transpositions on a given word by checking if each resulting word, obtained by 
        transposing two letters at a time, is present in the vocabulary. If a resulting word is found
        in the vocabulary, it calculates the edit count and returns a dictionary containing these 
        resulting words along with their corresponding edit counts.

        Parameters:
        - word: The input word for which transpositions need to be processed.

        Returns:
        - dict: A dictionary containing the resulting words after transposition and their corresponding edit counts.
        """ 
        counts = {}
        ## START YOU CODE HERE
        pass
        ## END
        return counts

    def process_replacements(self, word):
        """
        This function processes replacements on a given word by checking if each resulting word, obtained by 
        replacing a single letter at a time, is present in the vocabulary. If a resulting word is found
        in the vocabulary, it calculates the edit count and returns a dictionary containing these 
        resulting words along with their corresponding edit counts.

        Parameters:
        - word: The input word for which replacements need to be processed.

        Returns:
        - dict: A dictionary containing the resulting words after replacement and their corresponding edit counts.
        """ 
        counts = {}
        ## START YOU CODE HERE
        pass
        ## END
        return counts

    def read_edit_table(self, file_name):
        """
        Reads in the string edit counts file and stores it in a dictionary.

        Parameters:
        - file_name (str): Path to the edit counts file.

        Returns:
        - dict: A dictionary containing edit counts.
        """
        edit_table = {}
        with open(file_name, encoding='latin-1') as f:
            for line in f:
                contents = line.split("\t")
                edit_table[contents[0]] = int(contents[1])

        return edit_table

    def edit_count(self, s1, s2):
        """
        Returns how many times substring s1 is edited/modified/corrected as s2.

        Parameters:
        - s1 (str): Original substring.
        - s2 (str): Replacement substring.

        Returns:
        - int: The edit count.
        """
        return self.edit_table.get(s1 + "|" + s2, 0)


Test you code below:

In [ ]:
c = Corpus('data/trainset.txt')
model = ChannelModel(edit_file="data/count_1edit.txt", vocabulary=c.get_vocabulary())

word = 'read'
assert model.process_deletions(word) == {'red': 285}, 'Test failed'
print('Seccessful')
assert model.process_insertions(word) == {'ready': 8}, 'Test failed'
print('Seccessful')
assert model.process_replacements(word) == {'dead': 15, 'head': 4, 'lead': 45, 'road': 295, 'real': 4}, 'Test failed'
print('Seccessful')

word = 'there'
assert model.process_insertions(word) == {'theyre': 36, 'theres': 258}, 'Test failed'
print('Seccessful')
assert model.process_transpositions(word) == {'three': 189}, 'Test failed'
print('Seccessful')
assert model.process_replacements(word) == {'where': 3, 'these': 14}, 'Test failed'
print('Seccessful')

word = 'more'
assert model.process_insertions(word) == {'moore': 24}, 'Test failed'
print('Seccessful')
assert model.process_replacements(word) == {'fore': 2, 'sore': 3, 'move': 2}, 'Test failed'
print('Seccessful')

word = 'hello'
assert model.process_deletions(word) == {'hell': 18}, 'Test failed'
print('Seccessful')

word = 'dad'
assert model.process_insertions(word) == {'dead': 85, 'dads': 15}, 'Test failed'
print('Seccessful')
assert model.process_replacements(word) == {'bad': 37, 'had': 1, 'did': 559, 'day': 2}, 'Test failed'
print('Seccessful')

assert model.compute_edit_probabilities('read') == {'red': 0.04344512195121951, 'ready': 0.0012195121951219512, 
                                                    'dead': 0.0022865853658536584, 'head': 0.0006097560975609756, 
                                                    'lead': 0.006859756097560976, 'road': 0.04496951219512195, 
                                                    'real': 0.0006097560975609756, 'read': 0.9}, 'Test failed'
print('Seccessful')

assert model.compute_edit_probabilities('reda') == {'red': 0.1, 'reda': 0.9}, 'Test failed'
print('Seccessful')


## Language Models

Language models are a fundamental component of natural language processing and artificial intelligence, playing a crucial role in understanding and generating human-like text. These models are designed to comprehend the intricate patterns, structures, and semantics inherent in language, enabling machines to interact with and generate coherent and contextually relevant text.

At their core, language models are statistical and machine learning-based systems that learn the inherent rules and patterns of a language from vast amounts of textual data. Their primary objective is to predict the next word or sequence of words in a given context, harnessing the power of probabilistic relationships within a language. The ability to predict the likelihood of various word combinations empowers these models to capture syntactic, semantic, and contextual nuances, making them versatile tools for a wide array of applications.

Here, your task is to implement three Language Models.






### Uniform Language Model

### <span style="color:red"><b>Task 2</b></span>

Here your task is to build a simple uniform language model, which is capable of training on a given corpus and calculating the log-probability of sentences based on a uniform distribution of words.
Mathematically, for a sentence $s=w_1,w_2,w_3,\dots,w_n$ , the uniform language model score is calculated as follows:
$$\text{Uniform LM Score}(s) = log(P(s))=\sum_{w \in s } log(\frac{1}{|V|})$$
where $|V|$ is the size of the vocabulary.

In [ ]:
class UniformLanguageModel:

    def __init__(self, corpus):
        """
        Initialize your data structures in the constructor.
    
        Parameters:
        - corpus: The corpus to train the language model.
        """
        self.vocabulary = set()
        self.train(corpus)

    def train(self, corpus):
        """
        Takes a corpus and trains your language model.
        Compute any counts or other corpus statistics in this function.
  
        Parameters:
        - corpus: The corpus to train the language model.
        """
        ## START YOU CODE HERE
        pass
        ## END
    
    def score(self, sentence):
        """
        Takes a list of strings as an argument and returns the log-probability of the
        sentence using your language model. Use whatever data you computed in train() here.
    
        Parameters:
        - sentence (list): A list of strings representing the input sentence.
    
        Returns:
        - float: The log-probability of the sentence.
        """        
        score = 0.0
        ## START YOU CODE HERE
        pass
        ## END
        return score
        

Test your implementation here:

In [ ]:
"""Trains all of the language models and tests them on the dev data."""
trainPath = 'data/trainset.txt'
trainingCorpus = Corpus(trainPath)

uniformLM = UniformLanguageModel(trainingCorpus)
assert uniformLM.score(['I','am', 'Australian']) == -22.245525328839886, 'Test failed'
print('Seccessful')
assert uniformLM.score(['I','love','Deakin','University']) == -29.66070043845318, 'Test failed'
print('Seccessful')
assert uniformLM.score(['I','stud', 'CS','in','Australia']) == -37.07587554806648, 'Test failed'
print('Seccessful')
assert uniformLM.score(['I','am', 'enrolled', 'in', 'SIT770', 'as', 'a', 'graduate','student']) == -66.73657598651965, 'Test failed'
print('Seccessful')


### Laplace Unigram Language Model

### <span style="color:red"><b>Task 3</b></span>

Your task here is to implement a unigram language model using Laplace (add-one) smoothing.  The score calculated should be computed by summing the logarithms of Laplace-smoothed counts for each word in the sentence and adjusting for the total word count.
Mathematically, for a sentence $s=w_1,w_2,w_3,\dots,w_n$ , the Laplace Unigram Language Model score is calculated as follows:
$$\text{Laplace Unigram LM Score}(s)=log(P(s))=\sum_{w_i \in s } log(P(w_i))$$

where the **Laplace-smoothed unigram probability** $P(w_i)$ is calculated as:

$$P(w_i)=\frac{c(w_i)+1}{\sum_{ w_j\in V } c(w_j) +|V|}$$

where $|V|$ is the size of the vocabulary and $c(w_i)$ is the count indicating the number of times $w_i$ appears in the training set.
The Laplace smoothing involves adding 1 to both the count of $c(w_i)$  and the denominator (total count of all words plus the vocabulary size). This ensures that even words not observed in the training set have a non-zero probability.

In [ ]:
class LaplaceUnigramLanguageModel:

    def __init__(self, corpus):
        """Initialize your data structures in the constructor."""
        # TODO your code here
        self.total = 0
        self.vocabulary = set()
        self.LaplaceUnigramCounts = {}
        self.train(corpus)

    def train(self, corpus):
        """ Takes a corpus and trains your language model. 
            Compute any counts or other corpus statistics in this function.
        """
        ## START YOU CODE HERE
        pass
        ## END

    def score(self, sentence):
        """ Takes a list of strings as argument and returns the log-probability of the 
            sentence using your language model. Use whatever data you computed in train() here.
        """       
        score = 0.0
        ## START YOU CODE HERE
        pass
        ## END
        return score


Test your implementation here:

In [ ]:
"""Trains all of the language models and tests them on the dev data."""
trainPath = 'data/trainset.txt'
trainingCorpus = Corpus(trainPath)

laplaceUnigramLM = LaplaceUnigramLanguageModel(trainingCorpus)
assert laplaceUnigramLM.score(['I','am', 'Australian']) == -25.14565287682459, 'Test failed'
print('Seccessful')
assert laplaceUnigramLM.score(['I','love','Deakin','University']) == -35.57756036152766, 'Test failed'
print('Seccessful')
assert laplaceUnigramLM.score(['I','stud', 'CS','in','Australia']) == -42.56080392732966, 'Test failed'
print('Seccessful')
assert laplaceUnigramLM.score(['I','am', 'enrolled', 'in', 'SIT770', 'as', 'a', 'graduate','student']) == -68.26327621084289, 'Test failed'
print('Seccessful')


### Laplace Bigram Language Model

### <span style="color:red"><b>Task 4</b></span>

This LaplaceBigramLanguageModel class is designed to implement a bigram language model using Laplace (add-one) smoothing.  The train method processes the corpus, updating the bigram counts with Laplace smoothing. Similarly, the score method should calculate the log-probability of a given sentence based on the trained bigram language model, incorporating Laplace smoothing to handle unseen bigrams and unigrams. 
Mathematically, for a sentence $s=w_1,w_2,w_3,\dots,w_n$ , the Laplace Bigram Language Model score is calculated as follows:

$$
\begin{split}
\text{Laplace Bigram Language Model}(s) &=log(\prod_{w_i \in s } P(w_i))\\
&= log[P(w_1)P(w_2|w_1)P(w_3|w_2)\cdots P(w_n|w_{n-1})]\\
&= log(P(w_1))+log(P(w_2|w_1))+log(P(w_3|w_2))\cdots +log(P(w_n|w_{n-1}))]
\end{split}
$$

where the **Laplace-smoothed conditional probability** $P(w_i|w_{i-1})$ is calculated as:

$$
P(w_i|w_{i-1})=\frac{c(w_i,w_{i-1})+1}{ c(w_{i-1}) +|V|}
$$



where $|V|$ is the size of the vocabulary, and $c(w_i)$  and $c(w_i,w_{i-1})$ are respectively unigram and bigrams in the training set.

In [ ]:
class LaplaceBigramLanguageModel:

    def __init__(self, corpus):
        """
        Initialize your data structures in the constructor.

        Parameters:
        - corpus: The corpus to train the language model.
        """
        uniModel = LaplaceUnigramLanguageModel(corpus)
        self.LaplaceUnigramCounts = uniModel.LaplaceUnigramCounts
        self.LaplaceBigramCounts = {}
        self.total = uniModel.total
        self.train(corpus)

    def train(self, corpus):
        """
        Takes a corpus and trains your language model.
        Compute any counts or other corpus statistics in this function.

        Parameters:
        - corpus: The corpus to train the language model.
        """
        ## START YOUR CODE HERE
        pass
        ## END



    def score(self, sentence):
        """
        Takes a list of strings as an argument and returns the log-probability of the 
        sentence using your language model. Use whatever data you computed in train() here.

        Parameters:
        - sentence (list): A list of strings representing the input sentence.

        Returns:
        - float: The log-probability of the sentence.
        """
        score = 0.0
        ## START YOUR CODE HERE
        pass
        ## END
        return score


Test your implementation here:

In [ ]:
"""Trains all of the language models and tests them on the dev data."""
trainPath = 'data/trainset.txt'
trainingCorpus = Corpus(trainPath)

laplaceBigramLM = LaplaceBigramLanguageModel(trainingCorpus)
assert laplaceBigramLM.score(['I','am', 'Australian']) == -14.847658917530413, 'Test failed'
print('Seccessful')
assert laplaceBigramLM.score(['I','love','Deakin','University']) == -22.252126012871237, 'Test failed'
print('Seccessful')
assert laplaceBigramLM.score(['I','stud', 'CS','in','Australia']) == -29.747159786723298, 'Test failed'
print('Seccessful')
assert laplaceBigramLM.score(['I','am', 'enrolled', 'in', 'SIT770', 'as', 'a', 'graduate','student']) == -58.90693709044222, 'Test failed'
print('Seccessful')


## Spell Correction Channel using Noisy Channel Model

Now is time to consolidate all the components and assemble your SpellCorrect model, which comprises both the **Channel model** and the **Language Model**. The Channel model plays a pivotal role in estimating the likelihood of various corrections for observed misspelled words, while the Language Model contributes by evaluating the overall coherence and probability of entire sentences. By integrating these two crucial elements, your SpellCorrect model can intelligently correct spelling errors in a given text, making it a powerful tool for improving the accuracy and readability of textual content. 

### <span style="color:red"><b>Task 5</b></span>

Below, you will need to complete the `SpellCorrector` by generating the correct sentence. **The assumption made is that there is exactly one error in each sentence**.


In [ ]:
class SpellCorrector:
    """Holds edit model, language model, corpus. trains"""

    def __init__(self, lm, vocabulary):
        """Initializes the language model and edit model.

        Parameters:
        - lm (LanguageModel): The language model.
        - corpus: The corpus for training.
        """
        self.language_model = lm
        self.channel_model = ChannelModel('data/count_1edit.txt', vocabulary)

    def evaluate(self, corpus):
        """Tests this speller on a corpus, returns a SpellingResult.

        Parameters:
        - corpus: The corpus for evaluation.

        Returns:
        - SpellingResult: Result object containing correct and total counts.
        """
        numCorrect = 0
        numTotal = 0
        test_data = corpus.generate_test_cases()
        for sentence in test_data:
            if sentence.is_empty():
                continue
            errorSentence = sentence.get_error_sentence()
            hypothesis = self.get_likely_correct_sentence(errorSentence)
            if sentence.is_correction(hypothesis):
                numCorrect += 1
            numTotal += 1
        return SpellingResult(numCorrect, numTotal)

    def get_likely_correct_sentence(self, sentence):
        """Takes a list of words, returns a corrected list of words.

        Parameters:
        - sentence (list): List of words to correct.

        Returns:
        - list: Corrected list of words.
        """
        if len(sentence) == 0:
            return []
        argmax_i = 0
        argmax_w = sentence[0]
        maxscore = float('-inf')
        maxlm = float('-inf')
        maxedit = float('-inf')

        ## START YOUR CODE HERE
        pass
        ## END
        return argmax


Test your implementation here:

In [ ]:
"""Trains all of the language models and tests them on the dev data."""
trainPath = 'data/trainset.txt'
trainingCorpus = Corpus(trainPath)
laplaceBigramLM = LaplaceBigramLanguageModel(trainingCorpus)
laplaceBigramSpell = SpellCorrector(laplaceBigramLM, trainingCorpus.get_vocabulary())

assert laplaceBigramSpell.get_likely_correct_sentence(['I','lov','Australia']) == ['I', 'love', 'Australia'], 'Test failed'
print('Seccessful')



## Evaluation

It's time to subject your SpellCorrector to a comprehensive evaluation, leveraging the diverse language models you have implemented. This evaluation marks a crucial phase in assessing the efficacy and performance of your SpellCorrector across various linguistic scenarios. By applying the distinct language models, including, uniform LM, unigram LM, and bigram LM, you can systematically analyze how well your SpellCorrector handles different types of language complexities, spelling errors, and contextual nuances. 



In [ ]:
"""Trains all of the language models and tests them on the dev data."""
trainPath = 'data/trainset.txt'
trainingCorpus = Corpus(trainPath)

devPath = 'data/devset.txt'
devCorpus = Corpus(devPath)

print('Uniform Language Model: ')
uniformLM = UniformLanguageModel(trainingCorpus)
uniformSpell = SpellCorrector(uniformLM, trainingCorpus.get_vocabulary())
uniformOutcome = uniformSpell.evaluate(devCorpus)
assert uniformOutcome.get_accuracy() == 0.06581740976645435, 'UniformLanguageModel accuracy is incorrect'
print(str(uniformOutcome), '\n')

print('Laplace Unigram Language Model: ')
laplaceUnigramLM = LaplaceUnigramLanguageModel(trainingCorpus)
laplaceUnigramSpell = SpellCorrector(laplaceUnigramLM, trainingCorpus.get_vocabulary())
laplaceUnigramOutcome = laplaceUnigramSpell.evaluate(devCorpus)
assert laplaceUnigramOutcome.get_accuracy() == 0.11040339702760085, 'LaplaceUnigramLanguageModel accuracy is incorrect'
print(str(laplaceUnigramOutcome), '\n')


print('Laplace Bigram Language Model: ')
laplaceBigramLM = LaplaceBigramLanguageModel(trainingCorpus)
laplaceBigramSpell = SpellCorrector(laplaceBigramLM, trainingCorpus.get_vocabulary())
laplaceBigramOutcome = laplaceBigramSpell.evaluate(devCorpus)
assert laplaceBigramOutcome.get_accuracy() == 0.13588110403397027, 'LaplaceBigramLanguageModel accuracy is incorrect'
print(str(laplaceBigramOutcome), '\n')


### <span style="color:red"><b>Task 7</b></span>

To what extent did the outcomes of your constructed spelling correction system meet your initial expectations, and what valuable insights did you acquire from the system's performance?



# Congratulations!


Congratulations on completing the assignment! Your dedication and effort are commendable. By successfully working through the coding exercises and written exercises, you have demonstrated a strong understanding Language Modeling and it's application to Spelling Correction.


Congratulations on finishing this notebook! 




# Acknowledgement

## About the Author

This notebook was authored by Mohamed Reda Bouadjenek. He is a Senior Lecturer (Assistant Professor) of Applied Artificial Intelligence in the School of Information Technology at Deakin University, Australia.

## Contact Information

- **Email:** reda.bouadjenek@deakin.edu.au
- **GitHub:** https://github.com/rbouadjenek/

---
